In [15]:
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
cv_dir = '/media/zhaoyang-new/workspace/westonelison_finemap_simulator/CSE_284_Finemapping/simulations/archive_data/data/'
# cv_dir = '/media/zhaoyang-new/workspace/westonelison_finemap_simulator/CSE_284_Finemapping/simulations/simulation_summary_stats/'
finemap_folder = '/media/zhaoyang-new/workspace/westonelison_finemap_simulator/CSE_284_Finemapping/simulations/archive_data/finemapping_results/'

## Presentation Plots:

In [ ]:
data_folder = 'sampling_run/'

data = []
for folder in os.listdir(data_folder):
	rho_file = data_folder + '/' + folder + '/rhos.tsv'
	time_file = data_folder + '/' + folder + '/time.txt'

	meta_info = re.match(r"i(\d+)_K(\d+)_r(\d+)", folder.split('/')[-1])
	n_iteration = int(meta_info.group(1))
	k = int(meta_info.group(2))
	run = int(meta_info.group(3))

	snp_selected = []
	with open(rho_file) as fp_read:
		fp_read.readline()
		fp_read.readline()
		for line in fp_read:
			info = line.replace('\n', '').split('\t')
			snp_selected.append(int(info[1]))

	with open(time_file) as fp_read:
		line = fp_read.readline().replace('\n', '')
		time = float(line)

	data_entry = {'n_iteration': n_iteration,
				  'k': 'k=' + str(k),
				  'run': run,
				  'SNPs': snp_selected,
				  'n_SNP': len(snp_selected),
				  'time': time}
	data.append(data_entry)

df = pd.DataFrame(data)

In [ ]:
df

In [ ]:
df_grouped = df.groupby(['n_iteration', 'k'])['time'].agg(['mean', 'std']).reset_index()

fig, ax = plt.subplots(figsize=(12, 8))

for k_val in df_grouped['k'].unique():
    df_k = df_grouped[df_grouped['k'] == k_val]
    ax.errorbar(df_k['n_iteration'], df_k['mean'], yerr=df_k['std'], fmt='o-', label=f'{k_val}')

ax.set_xlabel('Number of Iterations', fontsize=14)
ax.set_ylabel('Time (sec)', fontsize=14)
ax.set_title('PyFM runtime vs. Number of Iterations', fontsize=18)
ax.legend(fontsize=14)
fig.savefig('figs/runtime.png')

In [ ]:
correct_SNPs = [153, 156, 166]

In [ ]:
count_correct_SNPs = lambda row: sum(snp in correct_SNPs for snp in row['SNPs'])
df['correct_SNPs_count'] = df.apply(count_correct_SNPs, axis=1)
df['percent_SNP_capture'] = df['correct_SNPs_count'] / 3.0

In [ ]:
df_grouped = df.groupby(['n_iteration', 'k'])['percent_SNP_capture'].agg(['mean', 'std']).reset_index()

fig, ax = plt.subplots(figsize=(12, 8))

for k_val in df_grouped['k'].unique():
    df_k = df_grouped[df_grouped['k'] == k_val]
    ax.plot(df_k['n_iteration'], df_k['mean'], label=f'{k_val}')

ax.set_xlabel('Number of Iterations', fontsize=14)
ax.set_ylabel('%Correct SNP Included', fontsize=14)
ax.set_title('PyFM %Causal Inclusion vs. Number of Iterations, Assuming 3 causal variants', fontsize=18)
ax.legend(fontsize=14)
fig.savefig('figs/correct_vs_nitr.png')

In [ ]:
df_filtered = df[(df['n_iteration'] == 1000) | (df['n_iteration'] == 700)]
df_grouped = df.groupby(['n_SNP', 'k'])['percent_SNP_capture'].agg(['mean', 'std']).reset_index()
# df_grouped = df_grouped[~((df_grouped['n_SNP'] == 11) & (df_grouped['k'] == 'k=5'))]
exhaustive = []
for i in range(2, 16):
	if i < 3:
		new_exhaustive = {'n_SNP': i, 'k': 'exhaustive', 'mean': i / 3, 'std': 0}
	else:
		new_exhaustive = {'n_SNP': i, 'k': 'exhaustive', 'mean': 1, 'std': 0}
	exhaustive.append(new_exhaustive)
df_grouped = df_grouped.append(exhaustive, ignore_index=True)

fig, ax = plt.subplots(figsize=(12, 8))

for k_val in df_grouped['k'].unique():
    df_k = df_grouped[df_grouped['k'] == k_val]
    ax.plot(df_k['n_SNP'], df_k['mean'], label=f'{k_val}')

ax.set_xlabel('Number of SNPs Selected', fontsize=14)
ax.set_ylabel('%Correct SNP Included', fontsize=14)
ax.set_title('PyFM %Causal Inclusion vs. Number of SNPs Selected, Assuming 3 causal variants', fontsize=18)

legend_order = ['k=3', 'k=5', 'k=7', 'exhaustive']  # Example order: k=1, k=3, k=5, exhaustive
handles, labels = plt.gca().get_legend_handles_labels()
ordered_handles = [handles[labels.index(label)] for label in legend_order]
ordered_labels = legend_order
plt.legend(ordered_handles, ordered_labels, loc='lower right', fontsize='large')

fig.savefig('figs/correct_vs_nsnp.png')

## Test PyFM on Simulated Data: convert file format

In [ ]:
in_file = '/media/zhaoyang-new/workspace/westonelison_finemap_simulator/CSE_284_Finemapping/simulations/simulation_summary_stats/1KGP_hg19_APOE_1MB.vcf.gz_173167643/ss_3_CV_10000_ctrl_10000_case.finemap.z'
df = pd.read_csv(in_file, sep=' ')

In [ ]:
out_dir = '/media/zhaoyang-new/workspace/PyFM/example/sim_test/'
df[['rsid', 'beta', 'se']].to_csv(out_dir + '173167643.z', index=False, header=False, sep=' ')

In [ ]:
out_dir = '/media/zhaoyang-new/workspace/PyFM/example/sim_test/'
df[['rsid', 'beta', 'se']].to_csv(out_dir + '173167643.z', index=False, header=False, sep=' ')

In [ ]:
## Extract True CV

In [ ]:
cv_data = []

for folder in [d for d in os.listdir(cv_dir) if os.path.isdir(os.path.join(cv_dir, d))]:
    region = folder.split('.')[0]
    seed = folder.split('_')[-1]
    
    folder_path = os.path.join(cv_dir, folder)
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    tsv_file_name = ""
    for f in files:
        if f.endswith('.tsv'):
            tsv_file_name = f
            break
    tsv_file_path = os.path.join(folder_path, tsv_file_name)
    tsv_df = pd.read_csv(tsv_file_path, sep='\t')
    cv_df = tsv_df[tsv_df['CV'] != 0]
    cv = cv_df['snps'].tolist()

    new_data_entry = {'region': region,
                      'seed': int(seed),
                      'CV': cv}
    cv_data.append(new_data_entry)

grand_cv_df = pd.DataFrame(cv_data)

In [ ]:
# grand_cv_df.to_csv('simulated_data_CV.csv', index=False)

## Extract PyFM Run Results

In [ ]:
def read_PyFM_output(data_folder):
    data = []
    for folder in os.listdir(data_folder):
        rho_file = data_folder + '/' + folder + '/rhos.tsv'
        
        if not os.path.isfile(rho_file):
            continue  # this folder's run was incomplete
    
        match = re.match(r"i(\d+)_K(\d+)_r(\d+)_cv(\d+)_region(\w+)_seed(\d+)", folder)
    
        
        i_value = int(match.group(1))
        K_value = int(match.group(2))
        r_value = int(match.group(3))
        cv_value = match.group(4)
        region_value = match.group(5)
        seed_value = int(match.group(6))
    
        snp_selected = []
        with open(rho_file) as fp_read:
            fp_read.readline()
            fp_read.readline()
            counter = 0
            for line in fp_read:
                info = line.replace('\n', '').split('\t')
                snp_selected.append(info[2])
            counter += 1
            if counter == 49:
                break
    
        new_data_entry = {'region': region_value,
                          'seed': seed_value,
                          'n_CV': cv_value,
                          'run_k': K_value,
                          'run_i': i_value,
                          'run_r': r_value,
                          'snp_selected': snp_selected}
        data.append(new_data_entry)
    df = pd.DataFrame(data)
    return df

In [ ]:
cv4_folder = '/media/zhaoyang-new/workspace/PyFM/simulated_data_run_cv4/'
cv4_df = read_PyFM_output(cv4_folder)

In [ ]:
cv5_folder = '/media/zhaoyang-new/workspace/PyFM/simulated_data_run_cv5/'
cv5_df = read_PyFM_output(cv5_folder)

## Extract FINEMAP Run Results

In [ ]:
data = []

for folder in os.listdir(finemap_folder):
    region = folder.split('.')[0]
    seed = folder.split('_')[-1]
    
    folder_path = os.path.join(finemap_folder, folder)
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    snp_file_name = ""
    for f in files:
        if f.endswith('.snp'):
            snp_file_name = f
            break
    snp_file_path = os.path.join(folder_path, snp_file_name)

    snp_df = pd.read_csv(snp_file_path, sep=' ')
    snp_selected = snp_df['rsid'].head(50).tolist()

    new_data_entry = {'region': region,
                          'seed': int(seed),
                          'snp_selected': snp_selected}
    data.append(new_data_entry)

finemap_df = pd.DataFrame(data)

## Get Correct Number of SNPs Called

In [ ]:
def count_cv(row, n_snp):
    return sum(1 for cv in row['snp_selected'][:n_snp] if cv in row['CV'])

In [ ]:
def get_nsnp_counts(a, b, df):
    df = pd.merge(df, grand_cv_df, on=['region', 'seed'], how='left')
    for i in range(a, b + 1):
        df['count_' + str(i)] = df.apply(count_cv, args=(i,), axis=1)
    return df

In [ ]:
cv4_df = get_nsnp_counts(4, 15, cv4_df)
cv5_df = get_nsnp_counts(4, 15, cv5_df)
finemap_df = get_nsnp_counts(4, 15, finemap_df)

In [ ]:
cv4_df

In [ ]:
cv5_df

In [ ]:
finemap_df

## Plotting for Paper

In [ ]:
pyfm_df = pd.concat([cv4_df, cv5_df], axis=0)

In [ ]:
pyfm_dict = {int(col.split('_')[1]): float(pyfm_df[col].mean()) for col in pyfm_df.columns if col.startswith('count_')}

In [ ]:
finemap_dict = {int(col.split('_')[1]): float(finemap_df[col].mean()) for col in finemap_df.columns if col.startswith('count_')}

In [ ]:
plt.figure(figsize=(10, 6))

plt.plot(list(pyfm_dict.keys()), list(pyfm_dict.values()), label='PyFM (k=4, 1k iterations)', marker='o')
plt.plot(list(finemap_dict.keys()), list(finemap_dict.values()), label='FINEMAP (k=5, 100k iterations)', marker='s')

# Add labels and title
plt.xlabel('Number of SNPs included in the final answer')
plt.ylabel('%Causal SNPs Caught')
plt.title('Number of SNPs included vs. %Causal Caught, with 4 Causal Variants simulated')
plt.legend()

# Show plot
# plt.show()
plt.savefig('paper_benchmark.png')

In [ ]:
df_grouped = df.groupby(['n_iteration', 'k'])['percent_SNP_capture'].agg(['mean', 'std']).reset_index()

fig, ax = plt.subplots(figsize=(12, 8))

for k_val in df_grouped['k'].unique():
    df_k = df_grouped[df_grouped['k'] == k_val]
    ax.plot(df_k['n_iteration'], df_k['mean'], label=f'{k_val}')

ax.set_xlabel('Number of Iterations', fontsize=14)
ax.set_ylabel('%Correct SNP Included', fontsize=14)
ax.set_title('PyFM %Causal Inclusion vs. Number of Iterations, Assuming 3 causal variants', fontsize=18)
ax.legend(fontsize=14)
fig.savefig('figs/correct_vs_nitr.png')

In [ ]:
df_grouped = df.groupby(['n_iteration', 'k'])['percent_SNP_capture'].agg(['mean', 'std']).reset_index()

fig, ax = plt.subplots(figsize=(12, 8))

for k_val in df_grouped['k'].unique():
    df_k = df_grouped[df_grouped['k'] == k_val]
    ax.plot(df_k['n_iteration'], df_k['mean'], label=f'{k_val}')

ax.set_xlabel('Number of Iterations', fontsize=14)
ax.set_ylabel('%Correct SNP Included', fontsize=14)
ax.set_title('PyFM %Causal Inclusion vs. Number of Iterations, Assuming 3 causal variants', fontsize=18)
ax.legend(fontsize=14)
fig.savefig('figs/correct_vs_nitr.png')